In [63]:
import torch
import numpy as np
import time
import torch.nn as nn
from torch.nn import functional as F
import mmap
import random
import pickle
device = "cuda" if torch.cuda.is_available() else "cpu"

In [61]:
block_size = 128
batch_size = 32
max_iters = 10000
learning_rate = 2e-5
eval_iters = 100
n_embed = 384
n_head = 8
n_layers = 8
dropout = 0.2

set:去重  
list:转化为列表  
sorted:排序  

In [12]:
from pathlib import Path

# the path of the directionary
dataset_path = Path("dataset")
# 拼接文件路径
# file_path = dataset_path / "wizard_of_oz.txt"
file_path = dataset_path / "vocab.txt"
print(file_path)

dataset/vocab.txt


In [14]:
with open(file_path,"r",encoding="utf-8") as f:
    text = f.read()
    chars = sorted(list(set(text)))
vocab_size = len(chars)
vocab_size

32172

In [17]:
str_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_str = {i:ch for i,ch in enumerate(chars)}

In [19]:
# 定义两个方法，将一个列表中的所有字母/数字都转化成数字/字母
encode = lambda s: [str_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_str[i] for i in l]) # 按照分隔符""将字符连接起来
decode2 = lambda l: [int_to_str[i] for i in l]
en_data = encode("hello")
en_data
de_data = decode(en_data)
de_data

'hello'

In [20]:
test1 = encode("hello")
print(test1)
test2 = decode(test1)
print(test2)
test3 =decode2(test1)
print(test3)

[74, 71, 78, 78, 81]
hello
['h', 'e', 'l', 'l', 'o']


In [22]:
data = torch.tensor(
    encode(text),
    dtype = torch.long
)
data[:100]

tensor([ 0,  1,  1,  1,  2,  1,  3,  1,  4,  1,  5,  1,  6,  1,  7,  1,  8,  1,
         9,  1, 10,  1, 11,  1, 12,  1, 13,  1, 14,  1, 15,  1, 16,  1, 17,  1,
        18,  1, 19,  1, 20,  1, 21,  1, 22,  1, 23,  1, 24,  1, 25,  1, 26,  1,
        27,  1, 28,  1, 29,  1, 30,  1, 31,  1, 32,  1, 33,  1, 34,  1, 35,  1,
        36,  1, 37,  1, 38,  1, 39,  1, 40,  1, 41,  1, 42,  1, 43,  1, 44,  1,
        45,  1, 46,  1, 47,  1, 48,  1, 49,  1])

In [24]:
len(text)

64344

In [26]:
n = int(0.8 * len(data))
train_data = data[:n]
val_data = data[n:]
print(len(train_data))
print(len(val_data))

51475
12869


In [28]:
length = len(train_data)

# def get_batch(split):
#     dataset = train_data if split == "train" else val_data
#     ix = torch.randint(0, len(dataset) - block_size, (batch_size,)) # 总共是几个batch
#     # a = [dataset[i:i+block_size] for i in ix] # 将每个batch构建成列表
#     x = torch.stack([dataset[i:i+block_size] for i in ix])  # 将每个batch中的数据进行堆叠，将列表中的元素进行堆叠
#     y = torch.stack([dataset[i+1:i+1+block_size] for i in ix])
#     x,y = x.to(device), y.to(device)
#     return x,y

mmap的优势体现在以下几点：  
- 高效读取处理大文件，避免内存爆炸
- 提供灵活的随机访问功能
- 对于get_random_chunk函数，mmap确保从任意位置高效的读取指定大小的数据块，而无需将整个文件加载到内存中

In [34]:
import os
os.getcwd()

'/data1/stu_Guoby/Learning/LLM/LLM_from_scrach'

In [35]:
# memory map for using small snippets of text from a single file of and size
def get_random_chunk(split):
    filename = "train_split.txt" if split == "train" else "val_split.txt"
    with open(filename, 'rb') as f:
        with mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ) as mm:
            # Determine the file size and a random position to start reading
            file_size = len(mm)
            start_pos = random.randint(0, (file_size) - block_size*batch_size)

            # seek to the random position and read the block of text
            mm.seek(start_pos)
            block = mm.read(block_size * batch_size -1)

            # Decode a block to a string, ignoring any invalid byte sequences
            # 将二进制数据转换为人类可读的字符串
            decoded_block = block.decode('utf-8', errors='ignore').replace('\r','')

            # train and test splits
            # 将字符串转变为Index，并且转换为tensor
            data = torch.tensor(encode(decoded_block), dtype=torch.long)
    return data

In [62]:
def get_batch(split):
    data = get_random_chunk(split)
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size]for i in ix])
    y = torch.stack([data[i+1:i+1+block_size]for i in ix])
    x,y = x.to(device), y.to(device)
    return x,y

In [43]:
x,y = get_batch('train')
print(f"input:\n{x}")
print(f"target:\n{y}")

input:
tensor([[79, 75, 73,  ...,  2, 78, 71],
        [75, 80, 77,  ..., 71, 80, 70],
        [67, 69, 77,  ...,  2, 82, 84],
        ...,
        [ 0,  0,  0,  ..., 68, 71, 18],
        [85, 74, 81,  ..., 75, 86, 75],
        [67, 78, 78,  ..., 67, 83, 14]], device='cuda:0')
target:
tensor([[75, 73, 74,  ..., 78, 71, 85],
        [80, 77, 71,  ..., 80, 70, 71],
        [69, 77, 70,  ..., 82, 84, 81],
        ...,
        [ 0,  0,  0,  ..., 71, 18, 68],
        [74, 81, 87,  ..., 86, 75, 69],
        [78, 78,  2,  ..., 83, 14,  2]], device='cuda:0')


In [45]:
# 取一个batch，查看输入和目标是什么
x = train_data[:block_size]
y = train_data[1: block_size + 1]
for i in range(block_size-1):
    context = x[0:i + 1]
    target = y[i]
    print(f"when the input is {context}, the target is {target}")

when the input is tensor([0]), the target is 1
when the input is tensor([0, 1]), the target is 1
when the input is tensor([0, 1, 1]), the target is 1
when the input is tensor([0, 1, 1, 1]), the target is 2
when the input is tensor([0, 1, 1, 1, 2]), the target is 1
when the input is tensor([0, 1, 1, 1, 2, 1]), the target is 3
when the input is tensor([0, 1, 1, 1, 2, 1, 3]), the target is 1
when the input is tensor([0, 1, 1, 1, 2, 1, 3, 1]), the target is 4
when the input is tensor([0, 1, 1, 1, 2, 1, 3, 1, 4]), the target is 1
when the input is tensor([0, 1, 1, 1, 2, 1, 3, 1, 4, 1]), the target is 5
when the input is tensor([0, 1, 1, 1, 2, 1, 3, 1, 4, 1, 5]), the target is 1
when the input is tensor([0, 1, 1, 1, 2, 1, 3, 1, 4, 1, 5, 1]), the target is 6
when the input is tensor([0, 1, 1, 1, 2, 1, 3, 1, 4, 1, 5, 1, 6]), the target is 1
when the input is tensor([0, 1, 1, 1, 2, 1, 3, 1, 4, 1, 5, 1, 6, 1]), the target is 7
when the input is tensor([0, 1, 1, 1, 2, 1, 3, 1, 4, 1, 5, 1, 6, 1, 7

In [46]:
a = torch.randint(0,4,(2,3,4))
print(a.shape)
w,t,c = a.shape
b = a.view(t,c,w)
print(b.shape)

torch.Size([2, 3, 4])
torch.Size([3, 4, 2])


In [48]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ["train","val"]:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X,Y = get_batch(split=split)
            _, loss = model(X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [50]:
class Head(nn.Module):
    """
    one head of self-attention
    """
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embed, head_size, bias=False)
        self.query = nn.Linear(n_embed, head_size, bias=False)
        self.value = nn.Linear(n_embed, head_size, bias=False)
        # 将tril注册为一个缓冲区，他们是模型的一部分，用作mask，但不会在训练中参与梯度更新
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))# 其实就是建立一个下三角矩阵

        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        # input of size: (batch, time-step, channels)
        # output of size: (batch, time-step, head_size)
        _,T,_ = x.shape
        k = self.key(x) # shape:(B,T,hs)
        q = self.query(x) # shape:(B,T,hs)
        # compute attention scores
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B,T,hs) * (B,hs,T) -> (B,T,T)
        # masked_fill:根据mask来填充张量中的某些位置，若是对应位置为True,则将该位置填充为指定的值
        # torch.tril():将某个张量转化为下三角矩阵，上三角部分置为0
        # 用-inf替换掉wei中与tril的上三角部分对应的元素
        wei = wei.masked_fill(
            self.tril[:T, :T] == 0,
            float('-inf')
        )
        wei = F.softmax(wei, dim=-1) # (B,T,T)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v
        return out

nn.ModuleList: 是一个容器，接受一个列表，并将该列表中的每个元素做为单独的子模块管理。跟nn.Sequential不太一样，它并不需要解包，它会将列表中的每个模块自动加入到ModuleList中。
nn.Sequential: 需要将每个子模块做为单独的参数传递给它，因此必须使用*进行解包列表。

In [52]:
class MultiHeadAttention(nn.Module):
    """
    multiple heads of self-attention in parallel
    """
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(num_heads * head_size, n_embed)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(out)
        return out

In [54]:
class FeedForward(nn.Module):
    """
    a simple linear layer followed by a non-linearity
    """
    def __init__(self, n_embed):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embed, 4*n_embed),
            nn.ReLU(),
            nn.Linear(4*n_embed, n_embed),
            nn.Dropout(dropout),
        )
    
    def forward(self, x):
        return self.net(x)


In [56]:
class Block(nn.Module):
    """
    Transformer Block: communication followed by computation
    """
    def __init__(self, n_embed, n_head):
        super().__init__()
        head_size = n_embed // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffd = FeedForward(n_embed)
        self.ln1 = nn.LayerNorm(n_embed)
        self.ln2 = nn.LayerNorm(n_embed)
    
    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x + y)
        y = self.ffd(x)
        x = self.ln2(x + y)
        return x

*是解包操作，将列表中的元素一个个进行解决出来，做为单独的参数传给函数
nn.Sequential()并不接受列表的输入，需要将其解包，然后一个个输入，一个个进行构建

In [58]:
class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
        self.position_embedding_table = nn.Embedding(block_size, n_embed)
        self.blocks = nn.Sequential(*[Block(n_embed, n_head) for _ in range(n_layers)])
        self.ln_f = nn.LayerNorm(n_embed)
        self.lm_head = nn.Linear(n_embed, vocab_size)
    
        self.apply(self._init_weights)
    
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)


    def forward(self, idx, targets=None):
        _,T = idx.shape
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)


        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, _ = self.forward(index)
            logits = logits[:, -1, :] # take the last time step
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)
        return index

In [59]:
model = GPTLanguageModel(vocab_size=vocab_size)
model = model.to(device)

# context = torch.zeros((1,1), dtype=torch.long, device=device)
# generated_chars = decode(model.generate(context, max_new_tokens=500)[0].tolist())
# print(generated_chars)

Adamw:
Adamw is a modification of the Adam optimizer that add weight decay to the parameter updates.
This helps to regularize the model and improve generalization performance.

In [60]:
optimizer = torch.optim.AdamW(
    model.parameters(), 
    lr = learning_rate
)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step{iter}, train loss: {losses['train']:.4f}, val loss: {losses['val']:.4f}")

    xb, yb = get_batch('train')
    logits, loss = model.forward(xb, yb)# 进行前向传播，只计算出了输出和损失，并未计算梯度

    optimizer.zero_grad(set_to_none=True) #优化之前将所有的梯度置为None
    loss.backward() # 计算梯度，反向传播误差
    optimizer.step() # 通过上面计算出来的梯度来更新模型的参数，正向传递信息
print(loss.item())

step0, train loss: 10.4565, val loss: 10.4547
step100, train loss: 6.6785, val loss: 6.7542
step200, train loss: 5.2323, val loss: 5.1719
step300, train loss: 4.0218, val loss: 4.0771
step400, train loss: 3.2853, val loss: 3.2483
step500, train loss: 2.8994, val loss: 3.0038
step600, train loss: 2.7195, val loss: 2.6882
step700, train loss: 2.6722, val loss: 2.5824
step800, train loss: 2.5961, val loss: 2.6235
step900, train loss: 2.5100, val loss: 2.5383
2.3116629123687744


In [64]:
with open("model/model-01.pkl","wb") as f:
    pickle.dump(model, f)
print("model saved")

model saved
